In [6]:
# Imports
import pandas as pd
import re
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import time

import requests
from bs4 import BeautifulSoup as soup
import lxml

import json
#from dateutil import rrule

## needs a py file to import properly in this structure
#from utils.collection_utils import datetime_parse

from itertools import combinations, permutations, chain


In [148]:
DATA_PATH = '../data/'
IN_DATA_PATH = '../data/input_data/'

### !!! Datetime function needs work

If the month dict makes a replacement, need to tag that as a month and limit my permutations as required

In [66]:
def datetime_parse(x):
      '''
      Parse datetime out of a string
      More functionality should be added as issues encountered 
      '''
      # Remove all non-alpha-numeric
      out = re.sub(r'[^0-9a-zA-Z:]+', ' ', x)

      ## Remove any starting tag with :
      # str.split('Updated: ', expand=True)
      
      # Remove time 
      # .str.replace(r'\b(([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9](:[0-5][0-9])?\s?([AaPp][Mm])?)', ' ')
      
      month_dict = {
      'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6',
      'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 
      'December':'12', 'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5',
      'Jun':'6', 'Jul':'7', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'
      }

      # Removing digits longer than 4 long and words not in months
      #out = [out.replace(key, value) for key, value in month_dict.items() if key in out][0]
      
      # Removing digits longer than 4 long and words not in months (now redundant)
      #out = re.sub(r'[0-9]\d{4,}', ' ', out)
      #out = re.sub(r'[0-9]+:[0-9]+', ' ', out) # all after : not tested

      out = out.split(' ')
      out = [word for word in out if word.isdigit() or word in list(month_dict.keys())]
      
      out = ' '.join(out[:3])
      # Strip loose whitespace

      out = out.strip()

      # Lists for parsing
      month = ['%b', '%m', '%B']
      day = ['%d']
      year = ['%Y', '%y']

      varieties = list(permutations(chain(month,day,year), 3))
      for v in varieties:
            v = ' '.join(v)
            try:
                  date = [datetime.strptime(str(out), v)]# if d != 0 else d for d in out] #old tag used elsewhere, here for ref.

                  if date is not None:
                        print('Successfully parsed with format: ' + v)
                        return date
                        break
            except:
                  #print('Failed: ' + v)
                  pass

In [23]:
# # To load as dataframe: no value at present 

# load_file = IN_DATA_PATH + 'collection_urls_df.jsonl'

# df = pd.read_json(load_file, convert_dates=True, lines=True, orient='records')
# df.head(7)

In [151]:
load_file = IN_DATA_PATH + 'collection_urls_dict.json'

with open(load_file) as handle:
    sources = json.loads(handle.read())

In [152]:
sources

{'CNA_Russia': 'https://www.cna.org/centers/cna/sppp/rsp/russia-ai-archive#newsletters',
 'CNA_China': 'https://www.cna.org/centers/cna/cip/china/china-ai-newsletter',
 'MIT_Technology_Review': 'https://www.technologyreview.com/',
 'Synced': 'https://syncedreview.com/',
 'IEEE_spectrum': 'https://spectrum.ieee.org/',
 'Import_AI': 'https://us13.campaign-archive.com/home/?u=67bd06787e84d73db24fb0aa5&id=6c9d98ff2c',
 'ChinAI': 'https://chinai.substack.com/archive?sort=new'}

In [5]:
# Get today's date
# To do: create a "last scraped json"
today = datetime.now()

today

datetime.datetime(2022, 4, 22, 9, 46, 7, 425081)

# Import AI

In [62]:
# Lists for saving collected data

title_list = []
url_list = []
dates = []


In [63]:
base_url = sources['Import_AI']

In [64]:
# Get list of all Import AI issues with date, title, and link

response = requests.post(base_url) #headers=header)

html = soup(response.text, 'lxml')

objects = html.find_all('li', class_="campaign")
for obj in objects:
      print(obj.text)
      print(obj.a['href'])
      title_list.append(obj.text)
      url_list.append(obj.a['href'])

print('------------')

04/11/2022 - Import AI 291: Google trains the world's biggest language model; how robots can be smarter about the world; Conjecture, a new AI alignment company
http://eepurl.com/hZjozT
04/05/2022 - Import AI 290: China plans massive models; DeepMind makes a smaller and smarter model; open source CLIP data
http://eepurl.com/hYSyi9
03/28/2022 - Import AI 289: Copyright v AI art; NIST tries to measure bias in AI; solar-powered Markov chains
http://eepurl.com/hYaLin
03/21/2022 - Import AI 288: Chinese researchers try to train 100trillion+ 'brain-scale' models; 33% of AI benchmarks are meaningless.
http://eepurl.com/hXCPbv
03/07/2022 - Import AI 287: 10 exaflop supercomputer; Google deploys differential privacy; humans can outsmart deepfakes pretty well
http://eepurl.com/hWsYwH
02/28/2022 - Import AI 286: Fairness through dumbness; planet-scale AI computing; another AI safety startup appears
http://eepurl.com/hVSMf1
02/21/2022 - Import AI 285: RL+Fusion; why RL demands better public policy;

In [65]:
# Parse date from list

for item in title_list:
      date_sequence = datetime_parse(item)
      dates.append(date_sequence[0])

04 11 2022 Import AI 291: Google trains the world s biggest language model how robots can be smarter about the world Conjecture a new AI alignment company
Successfully parsed with format: %m %d %Y
04 05 2022 Import AI 290: China plans massive models DeepMind makes a smaller and smarter model open source CLIP data
Successfully parsed with format: %m %d %Y
03 28 2022 Import AI 289: Copyright v AI art NIST tries to measure bias in AI solar powered Markov chains
Successfully parsed with format: %m %d %Y
03 21 2022 Import AI 288: Chinese researchers try to train 100trillion brain scale models 33 of AI benchmarks are meaningless 
Successfully parsed with format: %m %d %Y
03 07 2022 Import AI 287: 10 exaflop supercomputer Google deploys differential privacy humans can outsmart deepfakes pretty well
Successfully parsed with format: %m %d %Y
02 28 2022 Import AI 286: Fairness through dumbness planet scale AI computing another AI safety startup appears
Successfully parsed with format: %m %d %Y
0

In [100]:
# Save collection to dataframe

df_collected = pd.DataFrame(list(zip(title_list, url_list, dates)), 
            columns=['title', 'url', 'date'])

df_collected.head(2)

,url,title,date
0,http://eepurl.com/hZjozT,04/11/2022 - Import AI 291: Google trains the ...,2022-04-11
1,http://eepurl.com/hYSyi9,04/05/2022 - Import AI 290: China plans massiv...,2022-04-05


In [101]:
df_collected.columns

Index(['url', 'title', 'date'], dtype='object')

In [70]:
# Enter keywords in json referenced here

# For testing only
#search_terms = ['DeepMind', 'Google']

load_file = IN_DATA_PATH + 'collection_searchterms.json'

with open(load_file) as handle:
    search_terms = json.loads(handle.read())

In [72]:
search_terms['search_term']

['DeepMind', 'Google']

In [118]:
# List to save to
relevant_text = []

#preping list fo dictionary conversion
relevant_text.append(['title', 'url', 'date', 'text'])


In [119]:
# Gets text for results within timedelta window
#     This is probably best changed to a "last scraped" date from a json

for index, row in df_collected.iterrows():
      if row.date >= today - timedelta(days=30):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('p')#, class_="campaign")
            issue_text = []
            for obj in objects:
                  issue_text.append(obj.text)
                  #print(obj.text)
            issue_text = ' '.join(issue_text)
            #print(issue_text)
            #print('--------------------------------')
            
            for word in search_terms['search_term']:
                  if word in issue_text:
                        save = list(row)
                        save.append(issue_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
                  else: 
                        pass
            time.sleep(5)
      else:
            pass


Fetching: http://eepurl.com/hZjozT
True
Fetching: http://eepurl.com/hYSyi9
True
Fetching: http://eepurl.com/hYaLin
False
True


In [121]:
len(relevant_text)

4

In [122]:
## To save to json - not recommend as datetimes not serializable, would have to convert dt back to string
# Note - ugly dictionary, e.g. not jsonl or indented, but kept simple unless requested

# data_dict = {k: v for k, *v in zip(*relevant_text)}

# with open('data.json', 'w') as f:
#     json.dump(data_dict, f)

In [135]:
new_df = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [138]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'open_ai_data.csv')

In [140]:
combined_df = pd.concat([new_df, old_df])

In [145]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [146]:
combined_df

,url,title,date,text
0,http://eepurl.com/hZjozT,04/11/2022 - Import AI 291: Google trains the ...,2022-04-11 00:00:00,"Welcome to Import AI, a newsletter about artif..."
1,http://eepurl.com/hYSyi9,04/05/2022 - Import AI 290: China plans massiv...,2022-04-05 00:00:00,"Welcome to Import AI, a newsletter about artif..."
2,http://eepurl.com/hYaLin,03/28/2022 - Import AI 289: Copyright v AI art...,2022-03-28 00:00:00,"Welcome to Import AI, a newsletter about artif..."


In [137]:
combined_df.to_csv(DATA_PATH + 'open_ai_data.csv', index=False)

# Synced

In [153]:
base_url = sources['Synced']

'https://syncedreview.com/'

In [154]:
# just checking
today

datetime.datetime(2022, 4, 22, 9, 46, 7, 425081)

In [155]:
# Get date part for url
year = datetime.now().year
month = datetime.now().month

In [156]:
current_url = base_url + str(year) + '/' +  str(month) + '/'
current_url


'https://syncedreview.com/2022/4/'

In [191]:
# lists for saving
title_list = []
url_list = []
summary_list = []
date_list = []

In [192]:
# Setting up a fake useragent
# Can do for all collections, but for now limited to where is seems necessary only
# Ignore the error 

ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}
print(header)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}


In [193]:
response = requests.post(url, headers=header)

html = soup(response.text, 'lxml')

obj = html.find('div',id= "primary") 

In [195]:

dates = obj.find_all(class_='entry-date')
for d in dates:
      d_parsed = datetime_parse(d.text)
      date_list.append(d_parsed[0])
      #date_list.append(d.text)

titles = obj.find_all(class_='entry-title')
for t in titles:
      title_list.append(t.text)
      url_list.append(t.a['href'])

summaries = obj.find_all(class_='entry-summary')
for s in summaries:
      summary_list.append(s.text)



Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %

In [196]:
df_collected2 = pd.DataFrame(list(zip(title_list, url_list, date_list, summary_list)), 
            columns=['title', 'url', 'date', 'summary'])

df_collected2.tail(2)

,title,url,date,summary
38,Tsinghua & NKU’s Visual Attention Network Comb...,https://syncedreview.com/2022/02/23/deepmind-p...,2022-02-23,"\nIn the new paper Visual Attention Network, a..."
39,DeepMind Trains Agents to Control Computers as...,https://syncedreview.com/2022/02/22/deepmind-p...,2022-02-22,\nDeepMind trains agents to use keyboard and m...


In [ ]:
#keyword = 'AlexNet'
search_terms['search_term']

In [210]:
relevant_text = []

relevant_text.append(['title', 'url', 'date', 'summary', 'text'])


In [211]:
for index, row in df_collected2.iterrows():
      if row.date >= today - timedelta(days=14):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', class_="entry-content")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)

            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass


Fetching: https://syncedreview.com/2022/04/20/uc-berkeley-intels-photorealistic-denoising-method-boosts-video-quality-on-moonless-nights/
Fetching: https://syncedreview.com/2022/04/19/toward-self-improving-neural-networks-schmidhuber-teams-scalable-self-referential-weight-matrix-learns-to-modify-itself/
Fetching: https://syncedreview.com/2022/04/18/meet-deepdpm-no-predefined-number-of-clusters-needed-for-deep-clustering-tasks/
Fetching: https://syncedreview.com/2022/04/14/alibabas-usi-a-unified-scheme-for-training-any-backbone-on-imagenet-that-delivers-top-results-without-hyperparameter-tuning/
Fetching: https://syncedreview.com/2022/04/13/openais-unclip-text-to-image-system-leverages-contrastive-and-diffusion-models-to-achieve-sota-performance/
Fetching: https://syncedreview.com/2022/04/12/google-builds-language-models-with-socratic-dialogue-to-improve-zero-shot-multimodal-reasoning-capabilities/
Search term found: Google
Fetching: https://syncedreview.com/2022/04/11/maryland-u-google

In [212]:
new_df = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [213]:
new_df

,title,url,date,summary,text
0,Google Builds Language Models with Socratic Di...,https://syncedreview.com/2022/04/12/google-bui...,2022-04-12,\n In the new paper Socratic Models: Composing...,Large-scale language-based foundation models s...
1,Maryland U & Google Introduce LilNetX: Simulta...,https://syncedreview.com/2022/04/11/maryland-u...,2022-04-11,\nA team from the University of Maryland and G...,The current conventional wisdom on deep neural...


In [ ]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'synced_data.csv')

In [ ]:
combined_df = pd.concat([new_df, old_df])

In [ ]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [ ]:
combined_df

,url,title,date,text
0,http://eepurl.com/hZjozT,04/11/2022 - Import AI 291: Google trains the ...,2022-04-11 00:00:00,"Welcome to Import AI, a newsletter about artif..."
1,http://eepurl.com/hYSyi9,04/05/2022 - Import AI 290: China plans massiv...,2022-04-05 00:00:00,"Welcome to Import AI, a newsletter about artif..."
2,http://eepurl.com/hYaLin,03/28/2022 - Import AI 289: Copyright v AI art...,2022-03-28 00:00:00,"Welcome to Import AI, a newsletter about artif..."


In [ ]:
combined_df.to_csv(DATA_PATH + 'synced_data.csv', index=False)

# MIT

In [215]:
base_url = sources['MIT_Technology_Review']

In [260]:
topic_designator = 'topic/'
topic_tags = ['artificial-intelligence', 'humans-and-technology', 'computing'] 

In [248]:
final_url = base_url + topic_designator + topic_tags[0]
final_url

'https://www.technologyreview.com/topic/artificial-intelligence'

In [ ]:
for tag in topic_tags:
      final_url = base_url + topic_designator + tag
      # then search

In [254]:
response = requests.get(final_url)

# print headers of response
print(response.headers)

{'Server': 'nginx', 'Date': 'Fri, 22 Apr 2022 10:42:32 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '34115', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'ETag': 'W/"2ed7b-nH1JQs3vy3bcAMjKg3IzOTe7W+Y"', 'Cache-Control': 'max-age=300, must-revalidate', 'Content-Encoding': 'gzip', 'X-rq': 'hhn2 0 2 9980', 'Age': '0', 'X-Cache': 'miss', 'Vary': 'X-Mobile-Class, Accept-Encoding', 'Accept-Ranges': 'bytes', 'Strict-Transport-Security': 'max-age=31536000;includeSubdomains;preload'}


In [246]:
API_ROOT_URL = "https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name=artificial-intelligence&page=1"
'''
API_ROOT_URL
slug
?
requestType
=
query
&page=1'''

def searchApi():
    endpoint = API_ROOT_URL
    data = {
        "slug": "term_archive",
        "requestType": "category_name",
        "query": "artificial-intelligence"
    }
    try:
        response = requests.post(endpoint)#, data=data)
        if(response.status_code == 200):
            #print(response.json())
            for msg in response:
                print(msg)
    except Exception:
        print(traceback.format_exc())

In [253]:
searchApi()


In [266]:
API_Link = "https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name="
#artificial-intelligence"
#&page=1"

In [294]:
# Making a get test request
try:
      response = requests.get(API_Link)

      # print headers of response
      if(response.status_code == 200):
            print(response.headers)
            
except Exception:
      print(traceback.format_exc())


{'Server': 'nginx', 'Date': 'Fri, 22 Apr 2022 11:19:32 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '22', 'Connection': 'keep-alive', 'X-Robots-Tag': 'noindex', 'Link': '<https://wp.technologyreview.com/wp-json/>; rel="https://api.w.org/"', 'X-Content-Type-Options': 'nosniff', 'Access-Control-Expose-Headers': 'X-WP-Total, X-WP-TotalPages, Link', 'Access-Control-Allow-Headers': 'Authorization, X-WP-Nonce, Content-Disposition, Content-MD5, Content-Type', 'Access-Control-Allow-Origin': 'https://www.technologyreview.com', 'Cache-Control': 'max-age=60', 'Allow': 'GET', 'X-rq': 'hhn2 0 4 9980', 'Content-Encoding': 'gzip', 'Age': '0', 'X-Cache': 'miss', 'Vary': 'Accept-Encoding, Origin', 'Accept-Ranges': 'bytes', 'Strict-Transport-Security': 'max-age=31536000;includeSubdomains;preload'}


In [285]:
def get_response(API_Link, pages=1):
    with requests.get(API_Link + '&page=' + str(pages)) as response:
        page_soup = soup(response.content, 'lxml')
        return page_soup

In [286]:
responses = []

for tag in topic_tags:
      response = get_response(API_Link + tag, pages=str(1))

      #parse
      j_response = json.loads(response.text)
      responses = responses + j_response

In [297]:
responses[1].keys()

dict_keys(['name', 'config', 'children', 'componentGroups'])

In [308]:
responses[1]['config'].keys()

dict_keys(['excerpt', 'permalink', 'teaseCTA', 'title', 'topic', 'postDate', 'postFormat', 'topicLink', 'themeName', 'showImage', 'sponsorTagline', 'sponsorUrl', 'hideFeaturedImage', 'hideFooter', 'hideHeader', 'bodyLayout', 'storyColorTheme', 'headlineTextColor', 'headerBgColor', 'squareLeadImage', 'summaryBullets', 'subtopic', 'mitNewsTopic', 'mitNewsSubtopic', 'podcastLength', 'customEyebrow', 'byline', 'syndicate', 'hidePostContentAds', 'color'])

In [330]:
#initial empty df
collected_df = pd.DataFrame(columns=['title', 'url', 'date', 'summary'])
collected_df.loc[collected_df.index, :] = ['x', 'url', 'date', 'summary']

collected_df

,title,url,date,summary


In [333]:
needed_keys = ['title', 'permalink', 'postDate', 'excerpt']

entries = []
entries.append(['title', 'url', 'date', 'summary'])

for item in responses:
      entry = [item['config'][key] for key in needed_keys]
      entries.append(entry)
      #collected_df.loc[collected_df.index.max() + 1, :] = entry


In [336]:
new_df = pd.DataFrame(entries[1:],columns=entries[0])

In [366]:
def date_from_url(x):

      pat = r"(20[0-2][0-9]([-_/]?)[0-9]{2}(?:\2[0-9]{2})?)"
      dates = re.compile(pat)

      res = dates.search(x)

      res = datetime_parse(res[0])
  
      return res[0]


In [368]:
new_df['date'] = new_df['url'].apply(lambda x: date_from_url(x))


Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %

In [385]:
relevant_text = []

relevant_text.append(['title', 'url', 'date', 'summary', 'text'])


In [387]:
for index, row in new_df.iterrows():
      if row.date >= today - timedelta(days=7):
            print('Fetching: ' + row.url)
            response = requests.post(row.url, headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', id="content--body")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)

            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass


Fetching: https://www.technologyreview.com/2022/04/22/1050394/artificial-intelligence-for-the-people/
Search term found: Google
Fetching: https://www.technologyreview.com/2022/04/21/1050381/the-gig-workers-fighting-back-against-the-algorithms/
Fetching: https://www.technologyreview.com/2022/04/20/1050670/download-ai-capitalizes-catastrophe-bitcoin-cities-central-america-crypto/
Fetching: https://www.technologyreview.com/2022/04/20/1050392/ai-industry-appen-scale-data-labels/
Search term found: Google
Fetching: https://www.technologyreview.com/2022/04/19/1049592/artificial-intelligence-colonialism/
Search term found: Google
Fetching: https://www.technologyreview.com/2022/04/19/1049378/ai-inequality-problem/
Search term found: Google
Fetching: https://www.technologyreview.com/2022/04/21/1050815/hackers-target-critical-infrastructure-pwn2own/
Fetching: https://www.technologyreview.com/2022/04/21/1050747/cybercriminals-zero-day-hacks/
Fetching: https://www.technologyreview.com/2022/04/21/1

In [388]:
new_df2 = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [ ]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'synced_data.csv')

In [ ]:
combined_df = pd.concat([new_df, old_df])

In [ ]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [ ]:
combined_df.to_csv(DATA_PATH + 'mit_data.csv', index=False)

# IEEE

### Currently not working well - only 4 returns due to dynamic scrolling
I can solve with Selenium, but trying to avoid using it

Currently working on a requests version, but convoluated urls and html

In [572]:
responses = []

#pn = pagination
# when pn exceeds pages the html is quite short and maybe has a tag in it

url = "https://spectrum.ieee.org/res/load_more_posts/data.js?site_id=20265424&node_id=/root/blocks/block[search]/abtests/abtest[1]/row/column[1]/choose/otherwise/choose/otherwise/element_wrapper-&resource_id=search_deepfake&path_params={}&formats=html&q=deepfake&rm_lazy_load=1&pn=5&pn_strategy="


In [573]:
print(final_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.post(url, headers=header)

html = soup(response.text, 'lxml')


https://spectrum.ieee.org/search/?q=Google&order=newest


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [574]:
html

<html><body><p>{"path_params": {}, "node_id": "/root/blocks/block[search]/abtests/abtest[1]/row/column[1]/choose/otherwise/choose/otherwise/element_wrapper-", "exclude_post_ids": [], "search_phrase": "deepfake", "device": "desktop", "posts_html": "</p><div class='\"\"' id='\"sSearch_0_0_11_0_0_7_0_1_2_0_1_0\"'><div class='\"mb-2' id='\"sSearch_0_0_11_0_0_7_0_1_2_0_1_0_0\"' infinite="" story_teaser=""><div class='\"posts-custom' clearfix="" data-attr-all_element_order='\"all_sections,headline,subheadline,custom_field_PodcastEpisode,custom_field_PodcastVideoTime,date,time_to_read,like_button,custom_field_TimeofRead,tags,primary_tag,section,community_name,photo_caption,snark_line,author,post_shares,follow_button,community_comments,source_link,body,badges_sponsored,photo_credit,main_author,page_views,date_modified,custom_field_contact-person-phone,custom_field_event-performer,custom_field_event-street-address,custom_field_job-type,custom_field_event-postal-code,custom_field_contact-person-

In [ ]:
objects = html.find_all('div')#,  class_='section_column')#elid=True)#class_= 'clearfix')
print(len(objects))

for obj in objects:
      items = obj.find_all('h3')#, class_ = 'widget__head')#, id='col-right')
      print(len(items))
      for i in items:

            print(i.text)
            # title_list.append(i.h2.text)

            # print(i.h2.a['href'])
            # url_list.append(i.h2.a['href'])

            # date = i.div.span
            # print(date.text)
            # try: 
            #       #not working well
            #       d_parsed = datetime_parse(date.text)
            #       date_list.append(d_parsed[0])
            # except:
            #       date_list.append('None')


50
4
Commercialization of AI-assisted dubbing for TV and films hints at bigger possibilities with digitally captured acting performances


Facebook researchers release a dataset intended to help machine learning developers test algorithms for bias
4
Commercialization of AI-assisted dubbing for TV and films hints at bigger possibilities with digitally captured acting performances


Facebook researchers release a dataset intended to help machine learning developers test algorithms for bias
4
Commercialization of AI-assisted dubbing for TV and films hints at bigger possibilities with digitally captured acting performances


Facebook researchers release a dataset intended to help machine learning developers test algorithms for bias
4
Commercialization of AI-assisted dubbing for TV and films hints at bigger possibilities with digitally captured acting performances


Facebook researchers release a dataset intended to help machine learning developers test algorithms for bias
1
Commercializati

### Not great

In [390]:
base_url ='https://spectrum.ieee.org/'

In [391]:
query_url = 'https://spectrum.ieee.org/search/?q='

In [392]:
topic_url = 'https://spectrum.ieee.org/topic/artificial-intelligence/'

In [393]:
# query connector
C = "&"

In [394]:
sort_criteria = "order=newest"

In [395]:
topic_criteria = "topic=artificial-intelligence"

In [396]:
# initialized above
today

datetime.datetime(2022, 4, 22, 9, 46, 7, 425081)

In [397]:
today.date()

datetime.date(2022, 4, 22)

In [398]:
back_to = today.date() - timedelta(days=30)
back_to

datetime.date(2022, 3, 23)

In [400]:
search_terms['search_term']

['DeepMind', 'Google']

In [403]:
for term in search_terms['search_term']:
      final_url = query_url + term + C + sort_criteria
      print(final_url)

https://spectrum.ieee.org/search/?q=DeepMind&order=newest
https://spectrum.ieee.org/search/?q=Google&order=newest


In [555]:
print(final_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.post(final_url, headers=header)

html = soup(response.text, 'lxml')


https://spectrum.ieee.org/search/?q=Google&order=newest


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [434]:
title_list = []
url_list = []
date_list = []

In [435]:
# Works some at least but only fetches top 4 due to page scroll

objects = html.find_all('div',  class_='section_column')#elid=True)#class_= 'clearfix')

for obj in objects:
      items = obj.find_all('div', id='col-right')
      print(len(items))
      for i in items:

            print(i.h2.text)
            title_list.append(i.h2.text)

            print(i.h2.a['href'])
            url_list.append(i.h2.a['href'])

            date = i.div.span
            print(date.text)
            try: 
                  #not working well
                  d_parsed = datetime_parse(date.text)
                  date_list.append(d_parsed[0])
            except:
                  date_list.append('None')


4


        Video Friday: DALL-E 2
    

https://spectrum.ieee.org/video-friday-dall-e-2
08 Apr 2022
Successfully parsed with format: %m %b %Y


        How a Parachute Accident Helped Jump-start Augmented Reality
    

https://spectrum.ieee.org/history-of-augmented-reality
07 Apr 2022
Successfully parsed with format: %m %b %Y


        Volunteers Scramble to Preserve Ukraine’s Digital Culture
    

https://spectrum.ieee.org/archiving-ukraine-culture
06 Apr 2022
Successfully parsed with format: %m %b %Y


        How the Wayback Machine Is Saving Digital Ukraine
    

https://spectrum.ieee.org/internet-archive-ukraine
06 Apr 2022
Successfully parsed with format: %m %b %Y


In [436]:
df_collected3 = pd.DataFrame(list(zip(title_list, url_list, date_list)), 
            columns=['title', 'url', 'date',])

df_collected3.head(4)

,title,url,date
0,\n\n Video Friday: DALL-E 2\n \n,https://spectrum.ieee.org/video-friday-dall-e-2,2022-04-01
1,\n\n How a Parachute Accident Helped Ju...,https://spectrum.ieee.org/history-of-augmented...,2022-04-01
2,\n\n Volunteers Scramble to Preserve Uk...,https://spectrum.ieee.org/archiving-ukraine-cu...,2022-04-01
3,\n\n How the Wayback Machine Is Saving ...,https://spectrum.ieee.org/internet-archive-ukr...,2022-04-01


In [437]:
# List to save to
relevant_text = []

#preping list fo dictionary conversion
relevant_text.append(['title', 'url', 'date', 'text'])

In [438]:
today

datetime.datetime(2022, 4, 22, 9, 46, 7, 425081)

In [ ]:
for index, row in df_collected3.iterrows():
      if row.date >= today - timedelta(days=30):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', id="col-center")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)
   
            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass

In [443]:
len(relevant_text)

4

# CNA

In [ ]:
from  urllib.request import urlopen
import shutil

In [455]:
base_url_1 = 'https://www.cna.org/centers/cna/sppp/rsp/russia-ai-archive#newsletters'
base_url_2  = "https://www.cna.org/centers/cna/cip/china/china-ai-newsletters/issue-1"

In [456]:
# convert PDF 
# clean up PDF

In [457]:
today = datetime.now()

today

datetime.datetime(2022, 4, 22, 15, 37, 18, 711935)

In [458]:
back_to = today.date() - timedelta(days=30)
back_to

datetime.date(2022, 3, 23)

In [459]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
# create loop over base_url_1 and_2

In [522]:
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.get(base_url_2, headers=header)

html = soup(response.text, 'lxml')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [540]:
title_list = []
url_list = []
date_list = []

In [541]:

objects = html.find_all('div', id='newsletters')#, href=True)#elid=True)#class_= 'clearfix')
print(len(objects))

for obj in objects:
      items = obj.find_all('li')#, href=True)
      print(len(items))
      for i in items:
            print(i.text)
            title_list.append(i.text)
            urls = i.find_all('a', href=True)
            print(urls[1]['href'])
            url_list.append('https://www.cna.org' + urls[1]['href'])

            date = i.text
            try:
                  d_parsed = datetime_parse(date)
                  date_list.append(d_parsed[0])
            except:
                  date_list.append('None')

1
13
Issue 1: November 2, 2021 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-1.pdf
Successfully parsed with format: %B %m %Y
Issue 2: November 9, 2021 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-2.pdf
Successfully parsed with format: %B %m %Y
Issue 3: November 18, 2021 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-3.pdf
Successfully parsed with format: %B %d %Y
Issue 4: December 2, 2021 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-4.pdf
Successfully parsed with format: %B %m %Y
Issue 5: December 16, 2021 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-5.pdf
Successfully parsed with format: %B %d %Y
Issue 6: January 13, 2022 [ web | pdf ]
/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-6.pdf
Successfully parsed with format: %B %d 

In [542]:
# Save collection to dataframe

df_collected = pd.DataFrame(list(zip(title_list, url_list, date_list)), 
            columns=['title', 'url', 'date'])

df_collected.head(10)

,title,url,date
0,"Issue 1: November 2, 2021 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2021-02-01
1,"Issue 2: November 9, 2021 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2021-09-01
2,"Issue 3: November 18, 2021 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2021-11-18
3,"Issue 4: December 2, 2021 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2021-02-01
4,"Issue 5: December 16, 2021 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2021-12-16
5,"Issue 6: January 13, 2022 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2022-01-13
6,"Issue 7: January 27, 2022 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2022-01-27
7,"Issue 8: February 10, 2022 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2022-10-01
8,"Issue 9: February 24, 2022 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2022-02-24
9,"Issue 10: March 10, 2022 [ web | pdf ]",https://www.cna.org/CNA_files/centers/CNA/CIP/...,2022-10-01


### Get PDFs

In [545]:
QUERY = 'CNA'
pdf_dir = DATA_PATH + QUERY + 'pdfs'

if not os.path.exists(pdf_dir): 
  os.makedirs(pdf_dir)

have = set(os.listdir(pdf_dir))

# Time out for requests
timeout_secs = 10 


In [547]:
for index, row in df_collected.iterrows():
      if row.date >= today - timedelta(days=14):

            basename = row.url.split('/')[-1]
            fname = os.path.join(pdf_dir, basename)
            print(fname)

            # try:
            if not basename in have:
                  print('fetching %s into %s' % (row.url, fname))
                  req = urlopen(row.url, None, timeout_secs)
                  with open(fname, 'wb') as fp:
                        shutil.copyfileobj(req, fp)
            else:
                  print('%s exists, skipping' % (fname, ))


Fetching: https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-8.pdf
../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-8.pdf
fetching https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-8.pdf into ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-8.pdf
Fetching: https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-10.pdf
../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-10.pdf
fetching https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-10.pdf into ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-10.pdf
Fetching: https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-12.pdf
../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-12.pdf
fetching https://www.cna.org/CNA_files/centers/CNA/CIP/China/ai-newsletters/ChinaAI-Autonomy-Report-Issue-12.pdf into ../data/CNApdfs/ChinaAI-Autonomy-Report-Iss

### PDF to text

In [549]:
# Checking for a program and a folder
if not shutil.which('pdftotext'): # needs Python 3.3+
  print('ERROR: you don\'t have pdftotext installed. Install it first before calling this script')
  sys.exit()

if not os.path.exists(DATA_PATH + QUERY + '_txt'):
      os.makedirs(DATA_PATH + QUERY + '_txt')

In [550]:
# Specifying paths
txt_dir = DATA_PATH + QUERY + '_txt'
pdf_dir = DATA_PATH + QUERY + 'pdfs'

In [551]:
# Note- should make a function in utils and import

have = set(os.listdir(txt_dir))
files = os.listdir(pdf_dir)

for i,f in enumerate(files):

  txt_basename = f + '.txt'
  
  if txt_basename in have:
    print('%d/%d skipping %s, already exists.' % (i, len(files), txt_basename, ))
    continue

  pdf_path = os.path.join(pdf_dir, f)
  txt_path = os.path.join(txt_dir, txt_basename)
  
  cmd = "pdftotext %s %s" % (pdf_path, txt_path)
  os.system(cmd)

  print('%d/%d %s' % (i, len(files), cmd))

  # check output was made
  if not os.path.isfile(txt_path):
    # there was an error with converting the pdf
    print('there was a problem with parsing %s to text, creating an empty text file.' % (pdf_path, ))
    os.system('touch ' + txt_path) # create empty file, but it's a record of having tried to convert

  time.sleep(0.01) # allos for ctrl+c termination

0/6 skipping AI-and-autonomy-in-Russia-Issue-31.pdf.txt, already exists.
1/6 skipping AI-and-Autonomy-in-Russia-Issue-29-January-10-2022.pdf.txt, already exists.
2/6 pdftotext ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-8.pdf ../data/CNA_txt/ChinaAI-Autonomy-Report-Issue-8.pdf.txt
3/6 pdftotext ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-10.pdf ../data/CNA_txt/ChinaAI-Autonomy-Report-Issue-10.pdf.txt
4/6 pdftotext ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-12.pdf ../data/CNA_txt/ChinaAI-Autonomy-Report-Issue-12.pdf.txt
5/6 pdftotext ../data/CNApdfs/ChinaAI-Autonomy-Report-Issue-13.pdf ../data/CNA_txt/ChinaAI-Autonomy-Report-Issue-13.pdf.txt


In [ ]:
# To be nice I should parse these a bit and clean them up
# THEN open txt files and insert into DF (naturally create DF)

# See txts_to_csv notebook, easy task

# Scrap

In [468]:
def get_response():
    #base = test_link
    final_url = base_url
    #driver = webdriver.Chrome()
    
    with requests.get(final_url) as response:
        #response = requests.get(url, headers=headers)
        page_soup = soup(response.content, 'lxml')
        return page_soup

In [ ]:
from seleniumwire import webdriver  # Import from seleniumwire

# Create a new instance of the Firefox driver
#driver = webdriver.Chrome()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(chrome_options=chrome_options)
user_agent = driver.execute_script("return navigator.userAgent;")
# Go to the Google home page

driver.get(base_url_1)

# with driver.requests.get(base_url) as response:
#     #response = requests.get(url, headers=headers)
#     page_soup = soup(response.content, 'lxml')
#     #return page_soup

# Access requests via the `requests` attribute
for request in driver.requests:
    if request.response:
        print(
            request.path,
            request.response.status_code,
            request.response.headers['Content-Type'])


In [472]:
driver.close()
